### The report is created to evaluate products performance.

In [25]:
import pyodbc
import os
from IPython.core.display import display, HTML
from dotenv import load_dotenv
import pandas as pd

Connecting to SQL Server

In [26]:
load_dotenv('credentials.env')

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+os.environ.get('server')+';'
    'DATABASE='+os.environ.get('database')+';'
    'UID='+ os.environ.get('uid') +';'
    'PWD='+ os.environ.get('pwd')+';')


#### 1. Retrieve top 10 products with the highest total revenue.


In [27]:
top_ten_df = pd.read_sql("WITH revenue_by_product AS "
                         "(SELECT p.Name AS ProductName, pm.Name AS Model, "
                         "CONVERT(int, SUM(pod.LineTotal)) AS Revenue "
                         "FROM Purchasing.PurchaseOrderDetail pod "
                         "INNER JOIN Production.Product p "
                         "ON pod.ProductID = p.ProductID "
                         "INNER JOIN Production.ProductModel pm "
                         "ON pm.ProductModelID = p.ProductModelID "
                         "GROUP BY p.Name, pm.Name "
                         "ORDER BY SUM(pod.LineTotal) DESC "
                         "OFFSET 0 ROWS "
                         "FETCH FIRST 10 ROWS ONLY) "
                         "SELECT ProductName, Model, Revenue "
                         "FROM revenue_by_product "
                         "ORDER BY Revenue DESC;", conn)

def format(x):
    return "${:,.2f}".format(x)
top_ten_df['Revenue'] = top_ten_df['Revenue'].apply(format)

def plot_pandas_style(styler):
    html = '\n'.join([line.lstrip() for line in styler.render().split('\n')])
    return HTML(html)

display(top_ten_df)
plot_pandas_style

,ProductName,Model,Revenue
0,ML Mountain Pedal,ML Mountain Pedal,"$2,709,041.00"
1,ML Road Pedal,ML Road Pedal,"$2,390,330.00"
2,Front Brakes,Front Brakes,"$2,277,949.00"
3,Rear Brakes,Rear Brakes,"$2,277,949.00"
4,HL Mountain Tire,HL Mountain Tire,"$2,092,346.00"
5,ML Road Tire,ML Road Tire,"$1,866,376.00"
6,ML Mountain Tire,ML Mountain Tire,"$1,800,922.00"
7,Touring Pedal,Touring Pedal,"$1,766,855.00"
8,HL Mountain Pedal,HL Mountain Pedal,"$1,766,855.00"
9,LL Mountain Pedal,LL Mountain Pedal,"$1,766,561.00"


<function __main__.plot_pandas_style(styler)>


#### 2. Identify the products which where rejected by vendors, and the lost revenue due to returned products.


In [4]:
returned_products_df = pd.read_sql("SELECT sub.ProductName, "
                                   "sub.Model, "
                                   "CONVERT(int, SUM(RejectedQty)) AS TotalRejectedQty, "
                                   "CONVERT(int, SUM(LostRevenue)) AS TotalLostRevenue, "
                                   "sub.Vendor "
                                   "FROM "
                                   "(SELECT p.Name AS ProductName, "
                                   "pm.Name AS Model, "
                                   "pod.RejectedQty, "
                                   "pod.UnitPrice * pod.RejectedQty AS LostRevenue, "
                                   "v.Name AS Vendor "
                                   "FROM Purchasing.PurchaseOrderDetail pod "
                                   "INNER JOIN Production.Product p "
                                   "ON p.ProductID = pod.ProductID "
                                   "INNER JOIN Production.ProductModel pm "
                                   "ON pm.ProductModelID = p.ProductModelID "
                                   "INNER JOIN Purchasing.PurchaseOrderHeader poh "
                                   "ON pod.PurchaseOrderID = poh.PurchaseOrderID "
                                   "INNER JOIN Purchasing.Vendor v "
                                   "ON poh.VendorID = v.BusinessEntityID "
                                   "WHERE RejectedQty >= 1) sub "
                                   "GROUP BY sub.ProductName, sub.Model, sub.Vendor "
                                   "ORDER BY SUM(LostRevenue) DESC;;", conn)

def qty_format(x):
    return "{:,}".format(x)

def format(x):
    return "${:,.2f}".format(x)


returned_products_df['TotalRejectedQty'] = returned_products_df['TotalRejectedQty'].apply(qty_format)
returned_products_df['TotalLostRevenue'] = returned_products_df['TotalLostRevenue'].apply(format)

display(returned_products_df)

,ProductName,Model,TotalRejectedQty,TotalLostRevenue,Vendor
0,Rear Brakes,Rear Brakes,"1,430","$118,453.00",Superior Bicycles
1,Front Brakes,Front Brakes,"1,428","$118,287.00",Superior Bicycles
2,ML Road Pedal,ML Road Pedal,"1,375","$66,398.00",Mitchell Sports
3,ML Mountain Pedal,ML Mountain Pedal,"1,375","$66,398.00",Crowley Sport
4,Touring Pedal,Touring Pedal,"1,042","$65,635.00",Bicycle Specialists
5,ML Mountain Pedal,ML Mountain Pedal,"1,320","$63,742.00",Greenwood Athletic Company
6,ML Road Tire,ML Road Tire,"1,430","$56,111.00",Signature Cycles
7,HL Road Pedal,HL Road Pedal,825,"$51,966.00","Compete Enterprises, Inc"
8,LL Road Tire,LL Road Tire,"1,510","$51,861.00",Signature Cycles
9,LL Mountain Pedal,LL Mountain Pedal,"1,592","$50,131.00",Crowley Sport



#### 3. Retrieve the product Name and List Price (which is the product's selling price) for each product where the list price is higher than the unit price (vendor's selling price of a product) by 30% and more.


In [6]:
price_difference_df = pd.read_sql("SELECT sub.ProductName, "
                                  "sub.ListPrice AS SellingPrice, "
                                  "sub.UnitPrice AS VendorPrice, "
                                  "sub.PercentageDifference "
                                  "FROM "
                                  "(SELECT p.ProductID, p.Name AS ProductName, p.ListPrice, pod.UnitPrice, (p.ListPrice - pod.UnitPrice)/p.ListPrice * 100 AS PercentageDifference "
                                  "FROM Production.Product p "
                                  "INNER JOIN Purchasing.PurchaseOrderDetail pod "
                                  "ON pod.ProductID = p.ProductID) sub "
                                  "WHERE sub.ListPrice > sub.UnitPrice and PercentageDifference > 30 "
                                  "ORDER BY sub.PercentageDifference DESC, sub.ProductName;", conn)

def format(x):
    return "${:,.2f}".format(x)

def percentage_format(x):
    return "{}%".format(x)

price_difference_df['SellingPrice'] = price_difference_df['SellingPrice'].apply(format)
price_difference_df['VendorPrice'] = price_difference_df['VendorPrice'].apply(format)
price_difference_df['PercentageDifference'] = price_difference_df['PercentageDifference'].apply(percentage_format)

display(price_difference_df)

,ProductName,SellingPrice,VendorPrice,PercentageDifference
0,Headlights - Dual-Beam,$34.99,$4.43,87.33%
1,"Racing Socks, L",$8.99,$2.36,73.74%
2,"Racing Socks, M",$8.99,$2.36,73.74%
3,Hitch Rack - 4-Bike,$120.00,$33.88,71.76%
4,"Mountain Bike Socks, L",$9.50,$3.40,64.21%
5,"Mountain Bike Socks, M",$9.50,$3.40,64.21%
6,Bike Wash - Dissolver,$7.95,$2.97,62.64%
7,"Half-Finger Gloves, L",$24.49,$9.15,62.63%
8,"Half-Finger Gloves, M",$24.49,$9.15,62.63%
9,"Half-Finger Gloves, S",$24.49,$9.15,62.63%
